In [79]:
## analyse the data
import mne
import numpy as np
import matplotlib.pyplot as plt
import glob

from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [2]:
raw=mne.io.read_raw_gdf('BCICIV_2a_gdf/A01E.gdf',
                         eog=['EOG-left', 'EOG-central', 'EOG-right'])
raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])

Extracting EDF parameters from C:\Users\uabhi\NEW_MY_JUPYTER\BCI\BCI 4 dataset\BCICIV_2a_gdf\A01E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...


C:\Users\uabhi\anaconda3\envs\tf\lib\site-packages\mne\io\edf\edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
C:\Users\uabhi\anaconda3\envs\tf\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


<RawGDF | A01E.gdf, 22 x 687000 (2748.0 s), ~26 kB, data not loaded>

**From the paper**  
'1023': 1 (Rejected trial),   
 '1072': 2 (Eye movements),  
 '276':  3 (eyes open)),  
 '277':  4 (eyes closed),   
 '32766':5 (Start of a new run),  
 '768':  6 (Start of a trial),  
 '769': 7 (class 1),  
 '770': 8 (class 2),  
 '771': 9 (class 3),  
 '772': 10 (class 4)
#### event ids are not same for all subjects

In [3]:
def read_data(path):
    raw=mne.io.read_raw_gdf(path,preload=True,
                          eog=['EOG-left', 'EOG-central', 'EOG-right'])
    raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])
    raw.set_eeg_reference()
    events, event_id =mne.events_from_annotations(raw)
    #events[1]['769'],events[1]['770'],events[1]['771'],events[1]['772']
    ann = event_id.keys()
    ann = list(map(int,ann))
    ids = event_id.values()
    ids = list(ids)
    unq = np.unique(events[:,-1])
    for u in unq:
        events[:,-1] = np.where(events[:,-1]== u , ann[u-1],events[:,-1])
    epochs = mne.Epochs(raw, events, event_id=[769,772],
                        tmin= 0, tmax=4,baseline=(None,4), on_missing ='warn')
    labels=epochs.events[:,-1]
    features=epochs.get_data()
    return features,labels

In [4]:
paths = glob.glob('BCICIV_2a_gdf/*T.gdf')
paths

['BCICIV_2a_gdf\\A01T.gdf',
 'BCICIV_2a_gdf\\A02T.gdf',
 'BCICIV_2a_gdf\\A03T.gdf',
 'BCICIV_2a_gdf\\A04T.gdf',
 'BCICIV_2a_gdf\\A05T.gdf',
 'BCICIV_2a_gdf\\A06T.gdf',
 'BCICIV_2a_gdf\\A07T.gdf',
 'BCICIV_2a_gdf\\A08T.gdf',
 'BCICIV_2a_gdf\\A09T.gdf']

In [5]:
%%capture
features,labels,groups=[],[],[]
for i in range(len(paths)):
    feature,label=read_data(paths[i])
    features.append(feature)
    labels.append(label)
    groups.append([i+1]*len(label))

In [6]:
features=np.concatenate(features)
labels=np.concatenate(labels)
groups=np.concatenate(groups)

features.shape,labels.shape,groups.shape

((1296, 22, 1001), (1296,), (1296,))

In [7]:
groups

array([1, 1, 1, ..., 9, 9, 9])

In [8]:
unique, counts = np.unique(groups, return_counts=True)
unique, counts

(array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([144, 144, 144, 144, 144, 144, 144, 144, 144], dtype=int64))

In [9]:
unique, counts = np.unique(labels, return_counts=True)
unique, counts


(array([769, 772]), array([648, 648], dtype=int64))

# Tensor preparation

In [10]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [11]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [12]:
lb = preprocessing.LabelBinarizer()

#enc_df= enc.fit_transform(dataset['60'])
labels =  lb.fit_transform(labels)

In [13]:
np.unique(labels)

array([0, 1])

In [14]:
data_array = features
label_array = labels
group_array = groups
data_array=np.moveaxis(data_array,1,2)

accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
    break

In [34]:
scaler=StandardScaler3D()
train_features,val_features,train_labels,val_labels = train_test_split(data_array, label_array, test_size=0.2, random_state=42)
a,b,train_gr,val_gr = train_test_split(data_array, group_array, test_size=0.2, random_state=42)
train_features=scaler.fit_transform(train_features)
val_features=scaler.transform(val_features)

In [36]:
train_features.shape, train_labels.shape

((1036, 1001, 22), (1036, 1))

In [37]:
train_features=np.moveaxis(train_features,1,2)
val_features =np.moveaxis(val_features,1,2)
train_features.shape, train_features.shape

((1036, 22, 1001), (1036, 22, 1001))

# Spitting test data person by person

In [48]:
print(val_gr.shape)
print(val_features.shape)
print(val_labels.shape)
val_gr = np.array(val_gr)
print(val_gr)

(260,)
(260, 22, 1001)
(260, 1)
[5 3 6 8 9 7 4 5 7 1 6 8 6 4 7 1 1 6 5 3 3 7 8 1 4 2 9 2 6 3 9 3 6 6 5 1 7
 6 4 4 1 7 3 5 9 8 8 2 8 7 2 4 8 8 4 1 7 9 5 1 5 1 2 9 6 2 6 6 3 5 6 8 2 4
 7 7 8 8 4 6 7 4 7 6 1 6 9 5 1 6 5 9 5 3 2 2 6 2 3 2 1 7 2 4 2 7 5 3 2 3 2
 4 6 1 9 6 5 9 5 6 4 7 1 4 4 4 1 8 1 2 8 8 9 6 6 5 1 3 2 6 8 4 8 8 8 8 9 1
 4 4 1 3 2 3 2 1 4 2 3 7 9 2 6 3 4 1 1 2 4 6 7 2 6 9 9 7 1 8 8 8 7 9 9 3 6
 3 2 9 8 4 2 4 5 4 3 6 4 2 8 6 3 8 1 8 4 6 1 5 2 1 9 6 5 5 3 6 5 4 5 8 9 5
 5 9 8 9 7 5 8 7 9 7 3 5 8 8 7 2 7 9 3 3 7 2 6 3 1 7 3 4 5 8 5 3 2 6 6 2 1
 2]


In [51]:
unique, counts = np.unique(val_gr, return_counts=True)
unique, counts

(array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([28, 32, 27, 29, 27, 35, 26, 32, 24], dtype=int64))

In [71]:
def group_splitter(features, labels, groups):
    testing_features = []
    testing_labels = []
    for i in range(1,10):
        fe = []
        la = []
        for j in range(len(groups)):
            if groups[j]==i:
                fe.append(features[j])
                la.append(labels[j])
        testing_features.append(np.array(fe))
        testing_labels.append(np.array(la))
    return testing_features, testing_labels

In [72]:
testing_features, testing_labels = group_splitter(val_features,val_labels,val_gr)

1 2 3 4 5 6 7 8 9 

In [73]:
len(testing_features)

9

In [74]:
from EEGModels_binary_classifiers import *
import tensorflow as tf

# EegNet

In [158]:
model = tf.keras.models.load_model('Models/deepconvnet_LT_0.9038.h5')
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 22, 1001, 1)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 22, 1001, 8)       512       
                                                                 
 batch_normalization_9 (Batc  (None, 22, 1001, 8)      32        
 hNormalization)                                                 
                                                                 
 depthwise_conv2d_3 (Depthwi  (None, 1, 1001, 16)      352       
 seConv2D)                                                       
                                                                 
 batch_normalization_10 (Bat  (None, 1, 1001, 16)      64        
 chNormalization)                                                
                                                           

In [159]:
len(testing_features[1])

32

In [160]:
def find_accuracy(model,x,y):
    accuracy = []
    for i in range(len(x)):
        y_p = []
        pred = model.predict_on_batch(x[i])
        pred = pred.reshape(pred.shape[0])
        for l in pred:
            if l>0.5:
                y_p.append(1)
            else:
                y_p.append(0)
        y_t = y[i].reshape(y[i].shape)
        acc = accuracy_score(y_t, y_p)
        accuracy.append(acc)
    accuracy = np.array(accuracy)
    return np.array(accuracy)

In [161]:
accuracy = find_accuracy(model,testing_features,testing_labels)

In [162]:
accuracy

array([0.96428571, 0.8125    , 0.88888889, 0.86206897, 0.7037037 ,
       0.82857143, 0.88461538, 0.90625   , 1.        ])

In [163]:
unique, counts = np.unique(val_gr, return_counts=True)
unique, counts

(array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([28, 32, 27, 29, 27, 35, 26, 32, 24], dtype=int64))

In [164]:
accuracy.mean()

0.8723204539535957